**Plan**
* Try out popular pre-trained image models such as ResNet, VGG, and AlexNet?
* Add a dropout layer
* Plot graphs, try out tensorboard
* Reference: https://www.tensorflow.org/tutorials/images/transfer_learning

# Load Dataset

In [ ]:
!pip install -q tfds-nightly tensorflow-datasets tensorflow matplotlib

     |████████████████████████████████| 4.0 MB 3.8 MB/s 


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sklearn
import math
RANDOM_STATE = 4242
NUM_CLASSES = 9

In [ ]:
ds = tfds.load('deep_weeds', batch_size = -1, as_supervised= True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/17509 [00:00<?, ? examples/s]

Shuffling deep_weeds-train.tfrecord...:   0%|          | 0/17509 [00:00<?, ? examples/s]

Dataset deep_weeds downloaded and prepared to /root/tensorflow_datasets/deep_weeds/3.0.0. Subsequent calls will reuse this data.


In [ ]:
images, labels = ds['train'] # Type: EagerTensor
print(labels)

print(images.shape, labels.shape)

tf.Tensor([1 7 7 ... 8 8 6], shape=(17509,), dtype=int64)
(17509, 256, 256, 3) (17509,)


In [ ]:
images = tf.random.shuffle(images, seed=RANDOM_STATE)
labels = tf.random.shuffle(labels, seed=RANDOM_STATE)

In [ ]:
num_images = images.shape[0]
last_train_image = math.floor(num_images * 0.6)
last_val_image = math.floor(num_images * 0.8)

X_train = images[:last_train_image]
y_train = labels[:last_train_image]
X_val = images[last_train_image:last_val_image]
y_val = labels[last_train_image:last_val_image]
X_test = images[last_val_image:]
y_test = labels[last_val_image:]

print(X_train.shape, X_val.shape, X_test.shape)

(10505, 256, 256, 3) (3502, 256, 256, 3) (3502, 256, 256, 3)


In [ ]:
base_model = tf.keras.applications.ResNet50(include_top = False, weights = 'imagenet', input_shape = (256, 256, 3), pooling = None)

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
image_batch = X_train[:10]
label_batch = y_train[:10]
feature_batch = base_model(image_batch)
print(feature_batch)

tf.Tensor(
[[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]]

  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.43030090e-02
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [0.00000000e+00 0.00000000e+00 0.00000000e+

In [ ]:
# Frozen model outputs 2048 8x8 spatialfeatures
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_avg = global_avg_layer(feature_batch)
print(feature_batch_avg.shape)

(10, 2048)


In [ ]:
prediction_layer = tf.keras.layers.Dense(NUM_CLASSES, activation = 'softmax')
prediction_batch = prediction_layer(feature_batch_avg)
print(prediction_batch.shape)

(10, 9)


In [ ]:
# build the model
inputs = tf.keras.Input(shape = (256, 256, 3))
x = base_model(inputs, training = False)
x = global_avg_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 9)                 18441     
Total params: 23,606,153
Trainable params: 18,441
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
#Initial_loss
model.evaluate(X_val, y_val)

110/110 [==============================] - 33s 265ms/step - loss: 3.6857 - accuracy: 0.0503


[3.685739040374756, 0.05025699734687805]

In [ ]:
history = model.fit(x = X_train, y = y_train, epochs = 10)

Epoch 1/10
329/329 [==============================] - 90s 263ms/step - loss: 0.9231 - accuracy: 0.6925
Epoch 2/10
329/329 [==============================] - 86s 261ms/step - loss: 0.5891 - accuracy: 0.7992
Epoch 3/10
329/329 [==============================] - 86s 261ms/step - loss: 0.4904 - accuracy: 0.8311
Epoch 4/10
329/329 [==============================] - 86s 260ms/step - loss: 0.4379 - accuracy: 0.8538
Epoch 5/10
329/329 [==============================] - 86s 261ms/step - loss: 0.4002 - accuracy: 0.8643
Epoch 6/10
329/329 [==============================] - 86s 261ms/step - loss: 0.3584 - accuracy: 0.8813
Epoch 7/10
329/329 [==============================] - 86s 261ms/step - loss: 0.3387 - accuracy: 0.8869
Epoch 8/10
329/329 [==============================] - 86s 261ms/step - loss: 0.3176 - accuracy: 0.8945
Epoch 9/10
329/329 [==============================] - 86s 261ms/step - loss: 0.2990 - accuracy: 0.9013
Epoch 10/10
329/329 [==============================] - 86s 261ms/step - l

In [ ]:
model.evaluate(X_val, y_val)

110/110 [==============================] - 29s 259ms/step - loss: 0.5108 - accuracy: 0.8250


[0.5107982158660889, 0.8249571919441223]

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

In [24]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis=1) # Convert one-hot to index

[[6.68701436e-03 3.26166712e-02 3.08580715e-02 ... 1.74054150e-02
  1.52276084e-01 3.49118739e-01]
 [3.40377935e-03 1.97691494e-04 2.36918996e-09 ... 3.32615286e-08
  9.90776896e-01 5.61770098e-03]
 [5.21855072e-05 1.02865306e-04 1.79965296e-11 ... 7.11007277e-04
  1.78708520e-04 9.72008944e-01]
 ...
 [1.69124959e-07 3.48242284e-05 3.84305976e-10 ... 1.40092826e-09
  8.34009839e-08 9.99720156e-01]
 [2.03959877e-03 4.42437522e-05 1.87257028e-04 ... 2.15095997e-06
  2.75486382e-03 7.51265645e-01]
 [1.33862428e-04 1.22608922e-01 1.22737343e-09 ... 8.54878306e-01
  4.52586310e-03 1.68101247e-02]]


In [38]:
print(y_pred)
print(classification_report(y_val, y_pred))

[4 7 8 ... 8 8 6]
              precision    recall  f1-score   support

           0       0.88      0.47      0.61       210
           1       0.83      0.75      0.79       228
           2       0.90      0.85      0.88       220
           3       0.92      0.55      0.69       193
           4       0.84      0.74      0.79       222
           5       0.95      0.67      0.79       208
           6       0.90      0.78      0.84       235
           7       0.75      0.56      0.64       191
           8       0.80      0.96      0.87      1795

    accuracy                           0.82      3502
   macro avg       0.86      0.70      0.77      3502
weighted avg       0.83      0.82      0.82      3502

